# Abell 360 red sequence galaxies

Contact: Shenming Fu

Inspired by Céline Combet's Notebook

LSST Science Piplines version: Weekly 2025_17

Container Size: Medium

This notebook studies Red Sequence (RS) galaxies in the galaxy cluster Abell 360 (A360) using the ComCam data (DP1). The main steps are 

- Loading galaxies from object catalogs via Butler
- Making cuts on the galaxy sample
- Plotting Color-Magnitude Diagrams (CMD)
- Plotting Color-Color Plots (CC)
- Selecting RS galaxies
- Plotting the spatial distribution, radial profile, and magnitude distribution of RS galaxies 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import astropy.units as u
from astropy.coordinates import SkyCoord

from lsst.daf.butler import Butler
import lsst.geom as geom


### Load data

In [ ]:
repo = '/repo/main'
collections = 'LSSTComCam/runs/DRP/DP1/w_2025_09/DM-49235'
#collections = 'LSSTComCam/runs/DRP/DP1/w_2025_08/DM-49029'

butler = Butler(repo, collections=collections)
skymap = butler.get('skyMap', skymap='lsst_cells_v1')

In [ ]:
# Abell 360 BCG center
# https://www.legacysurvey.org/viewer?ra=37.8650&dec=6.9822&layer=ls-dr9&zoom=16
# z~0.22
ra_bcg = 37.865
dec_bcg = 6.982

# Given the center (ra,dec) and range, find covering tract+patch
range_deg = 0.1 

ra_min, ra_max = ra_bcg - range_deg, ra_bcg + range_deg
dec_min, dec_max = dec_bcg - range_deg, dec_bcg + range_deg

# Corners with units
radec = [geom.SpherePoint(ra_min, dec_min, geom.degrees),
         geom.SpherePoint(ra_min, dec_max, geom.degrees),
         geom.SpherePoint(ra_max, dec_min, geom.degrees),
         geom.SpherePoint(ra_max, dec_max, geom.degrees)]

tract_patch_list = skymap.findTractPatchList(radec)



In [ ]:
# https://pipelines.lsst.io/modules/lsst.source.injection/faqs/index.html
# The output tract_patch_dict will be a dictionary of overlapping tracts and patches. 
# Tract IDs are used as keys, with a list of patch IDs as values.

tract_patch_dict = {}
for tract_info, patch_info in tract_patch_list:
    tract_id = tract_info.tract_id
    patch_ids = [patch.sequential_index for patch in patch_info]
    tract_patch_dict[tract_id] = sorted(patch_ids)

print(tract_patch_dict)

### Color magnitude diagram and Color-color plot

In [ ]:
band_list = ['r', 'i', 'z']
merged_cat = pd.DataFrame()

for tract in tract_patch_dict:
    
    for patch in tract_patch_dict[tract]:
        
        dataId = {'tract': tract, 'patch' : patch ,'skymap':'lsst_cells_v1'}
        obj_cat = butler.get('objectTable', dataId=dataId)
        
        sel  = obj_cat['detect_isPrimary'] == True
        
        for band in band_list:
            sel &= obj_cat[f'{band}_cModel_flag'] == False
            sel &= obj_cat[f'{band}_cModelFlux'] > 0
    
        sel &= obj_cat['refExtendedness'] == 1

        merged_cat = pd.concat([merged_cat, obj_cat[sel]], 
                               ignore_index=True)


In [ ]:
c1 = SkyCoord(merged_cat['coord_ra'].values * u.deg, 
              merged_cat['coord_dec'].values * u.deg)
c2 = SkyCoord(ra_bcg*u.deg, dec_bcg*u.deg)
sep = c1.separation(c2)

sel_dist = sep.deg < range_deg 
merged_cat_s = merged_cat[sel_dist] 

In [ ]:
mag_dict = {}

for band in band_list:
    
    mag_dict[band] = -2.5 * np.log10(merged_cat_s[f'{band}_cModelFlux']) + 31.4
    mag_dict[f'{band}_snr'] = merged_cat_s[f'{band}_cModelFlux'] / merged_cat_s[f'{band}_cModelFluxErr']


In [ ]:
for band in band_list:

    fig, ax = plt.subplots(figsize=(4,3), layout='constrained')

    mag_lim_min, mag_lim_max = 15, 32
    bins = np.linspace(mag_lim_min, mag_lim_max, 41)
    mid = 0.5 * (bins[1:] + bins[:-1])
    
    count, _, _ = ax.hist(mag_dict[band], bins=bins, 
                          histtype='step')
    mid_max = mid[np.argmax(count)]
    ax.axvline(mid_max, c='r', ls='--', label='peak')
    
    ax.set_yscale('log')
    ax.set_xlabel('mag')
    ax.set_ylabel('count')
    ax.set_xlim([mag_lim_min, mag_lim_max])

    ax2 = ax.twinx()
    ax2.scatter(mag_dict[band], mag_dict[f'{band}_snr'], 
                s=1, c='k')
    ax2.axhline(5, c='g', ls=':', label='SNR=5')
    
    ax2.set_yscale('log')
    ax2.set_ylabel('SNR')

    plt.title(f'{band} band')

    ax.legend(loc="lower left")
    ax2.legend(loc="upper right")

    plt.savefig(f'mag_{band}.png')

In [ ]:
color_dict = {
    'ri': 0.5,
    'iz': 0.25,
}
eps = 0.1 
sel_rs = np.array([True] * len(merged_cat_s))
mag_max = 23

for ind in range(len(band_list)-1):

    band1 = band_list[ind]
    band2 = band_list[ind+1]

    fig, ax = plt.subplots(figsize=(4,3), layout='constrained')

    ax.scatter(mag_dict[band1], 
               mag_dict[band1] - mag_dict[band2], 
               marker='.', s=0.3)
    
    ax.set_xlim([18,25])
    ax.set_ylim([-2,2])
    
    ax.set_xlabel(band1)
    ax.set_ylabel(f'{band1}-{band2}')

    color = color_dict[f'{band1}{band2}']
    ax.axhline(color - eps, ls='--', c='k', alpha=0.3)
    ax.axhline(color + eps, ls='--', c='k', alpha=0.3)

    sel_rs &= np.abs(mag_dict[band1] - mag_dict[band2] - color) < eps
    sel_rs &= mag_dict[band1] < mag_max
    
    plt.savefig(f'{band1}_{band1}-{band2}.png')

In [ ]:
ind = 0
band1 = band_list[ind]
band2 = band_list[ind+1]
band3 = band_list[ind+2]

fig, ax = plt.subplots(figsize=(4,3), layout='constrained')

ax.scatter(mag_dict[band1] - mag_dict[band2], 
           mag_dict[band2] - mag_dict[band3], 
           marker='.', s=0.3)

ax.scatter(mag_dict[band1][sel_rs] - mag_dict[band2][sel_rs], 
           mag_dict[band2][sel_rs] - mag_dict[band3][sel_rs], 
           marker='.', s=1, label='RS')

ax.axvline(color_dict[f'{band1}{band2}'], ls='--', c='k', alpha=0.3)
ax.axhline(color_dict[f'{band2}{band3}'], ls='--', c='k', alpha=0.3)

ax.legend()

ax.set_xlim([-1,2])
ax.set_ylim([-1,2])

ax.set_xlabel(f'{band1}-{band2}')
ax.set_ylabel(f'{band2}-{band3}')

plt.savefig(f'{band1}-{band2}_{band2}-{band3}.png')

### Red sequence galaxies

In [ ]:
fig, ax = plt.subplots(figsize=(4,3), layout='constrained')
im = ax.scatter(merged_cat_s['coord_ra'][sel_rs], 
                merged_cat_s['coord_dec'][sel_rs], 
                s=2., 
                c=mag_dict['r'][sel_rs], 
                cmap='cool',
          )
ax.invert_xaxis()
ax.set_xlabel('ra [deg]')
ax.set_ylabel('dec [deg]')
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('r')

plt.savefig('rs_position.png')

#### Number density profile

In [ ]:
radial_distance = sep.arcmin[sel_dist][sel_rs]

bin_num = 6
bins = np.linspace(0, range_deg*60, bin_num)
mid = 0.5 * (bins[1:] + bins[:-1])
bin_size = bins[1] - bins[0]

count, _ = np.histogram(radial_distance, bins=bins)

fig, ax = plt.subplots(figsize=(4,3), layout='constrained')
ax.scatter(mid, count/(2*np.pi*mid*bin_size))
ax.set_xlabel('radial distance [arcmin]')
ax.set_ylabel('density [arcmin^-2]')
ax.set_yscale('log')

plt.savefig('rs_density_profile.png')

#### Magnitude histogram

In [ ]:
fig, ax = plt.subplots(figsize=(4,3), layout='constrained')

bins = np.linspace(15, mag_max, 21)

for band in band_list:
    ax.hist(mag_dict[band][sel_rs], bins=bins, 
            histtype='step', label=band)

ax.set_yscale('log')
ax.set_xlabel('mag')
ax.set_ylabel('count')
ax.legend()

plt.savefig('rs_mag_hist.png')

### Exercises

- Background galaxies (another NB):

  - Shape distribution
  - Mean shape profile

- Multi-patch cutout